In [1]:
import semantic_kernel as sk
import os
from dotenv import load_dotenv, find_dotenv
from helpers.LoggingHelper import LoggingHelper

_ = load_dotenv(find_dotenv()) # read local .env file
print(os.getenv("OPENAI_API_BASE"))

lh = LoggingHelper()
logger = lh.get_logger("semantic-kernel")


https://alkopenai2.openai.azure.com/


In [2]:

from semantic_kernel.connectors.ai.open_ai import (
    AzureChatCompletion,
    AzureTextCompletion,
)

kernel = sk.Kernel(log=logger)
kernel.add_chat_service(
    "chat_completion",
    AzureChatCompletion(
        "GPT35",
        os.getenv("OPENAI_API_BASE"),
        os.getenv("OPENAI_API_KEY")
    ),
)

In [3]:
# Now we need to import the plugin
from plugins.JarvisPlugin.Tasks import Tasks

# Now you can import the plugin importing skill directly from the function you declared
# in the plugin directory. The import_skill does not need the path, it only need an
# instance of the skill and the name of the skill
tasks_plugin = kernel.import_skill(Tasks(), skill_name="JarvisPlugin")


2023-11-29 09:21:09,875 - semantic-kernel - DEBUG - Importing skill JarvisPlugin
2023-11-29 09:21:09,877 - semantic-kernel - DEBUG - Methods imported: 1


In [4]:
from pprint import pprint
# want to print all the keys of extractaudio_plugin that is a dictionary

pprint (tasks_plugin["ChangeTaskTitle"])

In [5]:
# We start using a planner, that allows us not to have to manually call functions
from semantic_kernel.planning.basic_planner import BasicPlanner
planner = BasicPlanner()

In [6]:
# now we can use the planner
question = "I want to change title of Task_34 to 'Hello semantic kernel'"
plan = await planner.create_plan_async(question, kernel)

result = await planner.execute_plan_async(plan, kernel)

print(f"Plan results: {result}")

print("------------------")
print(plan.generated_plan)

2023-11-29 09:21:20,928 - semantic-kernel - DEBUG - Extracting blocks from template: 
You are a planner for the Semantic Kernel.
Your job is to create a properly formatted JSON plan step by step, to satisfy the goal given.
Create a list of subtasks based off the [GOAL] provided.
Each subtask must be from within the [AVAILABLE FUNCTIONS] list. Do not use any functions that are not in the list.
Base your decisions on which functions to use from the description and the name of the function.
Sometimes, a function may take arguments. Provide them if necessary.
The plan should be as short as possible.
For example:

[AVAILABLE FUNCTIONS]
EmailConnector.LookupContactEmail
description: looks up the a contact and retrieves their email address
args:
- name: the name to look up

WriterSkill.EmailTo
description: email the input text to a recipient
args:
- input: the text to email
- recipient: the recipient's email address. Multiple addresses may be included if separated by ';'.

WriterSkill.Transla

Now call the api to change the title of the task Task_34 to Hello semantic kernel
Plan results: 7
------------------
{
    "input": "Change task title",
    "subtasks": [
        {"function": "JarvisPlugin.ChangeTaskTitle", "args": {"new_title": "Hello semantic kernel", "task_id": "Task_34"}}
    ]
}
